In [1]:
%matplotlib

Using matplotlib backend: Qt5Agg


In [6]:
import sys, os
ATS_SRC_DIR1 = 'S:/ats86/repos/ats'

sys.path.append(os.path.join('S:/ats86/repos/ats/tools/meshing_ats')) #os.environ['ATS_SRC_DIR1'],'tools','meshing_ats'))
print(sys.path)
import meshing_ats
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import math

['', 'C:\\Users\\mtoconnor12\\Anaconda2\\python27.zip', 'C:\\Users\\mtoconnor12\\Anaconda2\\DLLs', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\plat-win', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\lib-tk', 'C:\\Users\\mtoconnor12\\Anaconda2', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\site-packages', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\site-packages\\win32', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\site-packages\\win32\\lib', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\site-packages\\Pythonwin', 'C:\\Users\\mtoconnor12\\Anaconda2\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\mtoconnor12\\.ipython', 'S:/mo8557/ats86/repos/ats/tools/meshing_ats', 'S:/ats86/repos/ats/tools/meshing_ats', 'S:/ats86/repos/ats/tools/meshing_ats', 'S:/ats86/repos/seacas/lib', 'S:/ats86/repos/ats/tools/meshing_ats', 'S:/ats86/repos/seacas/lib', 'S:/ats86/repos/ats/tools/meshing_ats']


WindowsError: [Error 126] The specified module could not be found

In [ ]:
xl =pd.ExcelFile('Topo_Profiles_Barrow.xlsx')
xl.sheet_names
print (xl.sheet_names)
data = xl.parse(sheet_name='Transect B, Area C')
keys = data.keys()
#plt.plot(data['Distance_m'], data['Z'])
#plt.ylabel('Elevation')
#plt.title('Topography, Polygon Area C')
#plt.show()

# smooth the topographic data

# To get topography from a function, define it here, e.g., z = sine(x)
x = np.array(data['Distance_m'][:])
z = np.array(data['Z'][:])
    
# If you topographic data is already smooth then you may not want to run this savgol_filter. 
import scipy.signal
z_smooth = scipy.signal.savgol_filter(z, window_length=11, polyorder=3, mode='nearest')


plt.plot(x, z, '.')
plt.plot(x, z_smooth, 'k-')
plt.ylabel('Elevation')
plt.title('Mesh, Polygon Area C')
plt.show()

In [ ]:
# From a fucntion
# To get topography from a function, define it here, e.g., z = sine(x)
A = 0.15
k1 = math.pi
k2 = math.pi
m = 0.1

x = np.linspace(0,2,101)
y = np.linspace(0,2,101)


[X,Y] = np.meshgrid(x,y)

Z = A*np.sin(k1*X)*np.sin(k2*Y) + m*X


plt.contourf(X,Y,Z)



In [ ]:
""" 
# 2D Hillslope from a fucntion
# Topography from a function: e.g., z = sine(x)

x = np.linspace(0,500,501)
x0,z0 = 0, 5
z = z0 + 0.25 * (x - x0)

plt.plot(x, z, '.')
plt.title('Mesh, Polygon Area C')
plt.show()
m2 = meshing_ats.Mesh2D.from_Transect(x,z)
"""

In [ ]:
import meshing_ats
m2 = meshing_ats.Mesh2D.from_2DSurface(x,y,Z) #1D x, 1D y, and 2D Z
#m2.plot()

In [ ]:

# layer extrusion
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []
n1,n2 = 4, 20 #8cm peat 
#n1, n2 = 10, 14 #20cm peat

for i in range(2):
    layer_types.append('constant')
    layer_data.append(0.01)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.01
    Z.append(z)
print (z)

for i in range(n1): #8cm peat n=4, 20cm peat n=10
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print (z)  
for i in range(n2): #8cm peat, n=20, 20cm peat n = 14
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print (z)
dz = .02
for i in range(21):
    dz *= 1.05
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    #print ('3rd layer',z)
    z = z + dz
    Z.append(z)
print (z)

for i in range(17):
    dz *= 1.2
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    #print ('3rd layer',z)
    z = z + dz
    Z.append(z)
print (z)


for i in range(8):
    dz *= 1.2
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    #print ('5th layer',z)
    z = z + dz
    Z.append(z)
print (z)
#layer_types.append('snapped') #snapped is to flat the bottom
#layer_data.append(-40.0) # bottom location
#layer_ncells.append(2) # these two cells will make the bottom at 40 m
#layer_mat_ids.append(1003)

m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)
m3.write_exodus("testgrid.exo")
#m3.write_exodus("column-peat20cm.exo")

In [ ]:
print 2/101.

In [ ]:
0.08/8

In [ ]:
0.02
0.1
0.5
1.25010428797
8.33294983235
32.8082236775